## Finding the Needle in the TAD-stack: Predicting Topologically Associated Domain (TAD) Boundaries in Healthy and Cancer Cell Genomes

### 02710 - Computational Genomics
### Evan Corden, Sanat Mishra, Vallari Shende

Topologically Associated Domains (TADs) are genomic regions with a high density
of interactions between DNA segments within the same region and low interactions
with segments in neighboring regions. Alterations in TAD structure and function
have been found to be linked to several diseases such as cancer, neurodegenerative
disorders, and developmental disorders. TAD boundaries are regions flanking
TADs that are highly conserved within different mammalian cell types as well as
across species. Some transfer RNA and housekeeping genes in TADs appear to be
near the TAD boundary more often than expected by random chance. Therefore,
a disruption in a TAD boundary will lead to some genes being excluded from
the TAD, which can be associated with changes in expression of those and other
genes. Such disruptions are known to be associated with cancer development and
progression. In this project, we aim to build a machine learning model to predict
the TAD boundaries in healthy breast epithelial cells (GM12878) using genomic
features from ChIP-seq and ATAC-seq data

In [ ]:
import os
import subprocess
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score,roc_auc_score,f1_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [ ]:
#Provide data file paths

#True TAD boundaries for the respective cell type
true_tad_boundaries=
#"/Users/sanatmishra27/Downloads/CompGenomics/GM12878/GM12878_TAD.bed"

#Chromosome sizes 
chrom_sizes=
#"/Users/sanatmishra27/Downloads/CompGenomics/02710_Project/hg38.chrom.sizes"

#The folder containing the chip-seq files in bigbed format for the respective cell type
chip_file_dir=
#"/Users/sanatmishra27/Downloads/CompGenomics/GM12878/ChIP-seq/"

#The atac-seq data file for the respective cell type
atac_seq_data=#"/Users/sanatmishra27/Downloads/CompGenomics/GM12878/GM12878_ATAC.bed"

#Can be inside the respective cell line folder (not a strict requirement)
chip_seq_output=
#"/Users/sanatmishra27/Downloads/CompGenomics/GM12878/Data_Matrix_New.txt"

#### Binning genomic data for use with ChIP-seq and ATAC-seq data

In [ ]:
#The genomic data from all chromosomes were binned into 10000 base regions


chromosome_size = {} # Dictionary with key equal to chr and val = size
bin_size = 10000

# Get total chromosome size
with open(chrom_sizes, "r") as in_file:
    for line in in_file:
        line_array = line.strip().split("\t")
        
        chr = line_array[0]
        if len(chr) > 5:
            continue
        if chr == "chrM":
            continue
        if chr == "chr23" or chr == "chrX" or chr == "chrY":
            continue
        size = int(line_array[1])
        chromosome_size[chr.strip()] = [size]

chromosome_names = []

# only using 22 chromosome to fit the TAD data
for i in range(1,23):
    chromosome_names.append(str(i))

# Save cumulative chromosome lengths
prev_cum_sum = 0
for i in chromosome_names:
    size = chromosome_size["chr" + i][0]
    chromosome_size["chr" + i].append(prev_cum_sum)
    prev_cum_sum += size

total_genome_length = 0
for i in chromosome_names:
    total_genome_length += chromosome_size["chr" + i][0]

num_bins = int(total_genome_length / bin_size) + 1

#### Reading in TAD boundaries (true labels)

In [ ]:
# Import TAD regions for the corresponding cell line

tad_chr = []
tad_co = []
tad_bo_co = []

tad_boundary_offset = 100000


with open(true_tad_boundaries, "r") as in_file:
    
    if 'GM12878' in true_tad_boundaries:
        for line in in_file:
            line_array = line.strip().split("\t")
            chr = line_array[0]
            start = int(line_array[1])
            stop = int(line_array[2])

            if chr == "chrX":
                continue
            if start < tad_boundary_offset:
                tad_bo_co.append([-1, -1, stop, stop + tad_boundary_offset])
            elif stop > chromosome_size[chr][0]:
                tad_bo_co.append([start - tad_boundary_offset, start, -1, -1])
            else:
                tad_bo_co.append([start - tad_boundary_offset, start, stop, stop + tad_boundary_offset])

            tad_chr.append(chr)
            tad_co.append([start, stop])
    else:
        for line in in_file:
            line_array = line.strip().split("\t")
            chr = line_array[0]
            start = int(line_array[1])
            stop = int(line_array[2])

            if chr == "chrX":
                continue
            tad_bo_co.append([start, stop])

            tad_chr.append(chr) 
        


#### Reading in ChIP-seq data

In [ ]:
#The ChIP-seq files were downloaded from ENCODE in the bigBED format

directory=chip_file_dir
get_files=os.listdir(directory)

#Then, these files were converted into the BED format for downstream use

gene_count=0
for file in get_files:
    if 'bigBed' in file:
        file_details=file.split('bigBed')
        name=file_details[0]
        subprocess.run(['bigBedtoBed',directory+'/'+file,directory+'/'+name+'bed'])
        gene_count+=1

In [ ]:
# Import ChIP-seq data from BED files

path=chip_file_dir
chip_chr = []
chip_co = []
chip_sig = []
fp=os.listdir(path)

for i in fp:
    if 'bed' in i:
        chip_chr_d=[]
        chip_co_d = []
        chip_sig_d = []
        with open(path+i, "r") as in_file:
            for line in (in_file):
                line_array = line.strip().split("\t")
                chr = line_array[0]
                start = int(line_array[1])
                stop = int(line_array[2])
                sig = float(line_array[6])

                chip_chr_d.append(chr)
                chip_co_d.append([start, stop])
                chip_sig_d.append(sig)
        chip_chr.append(chip_chr_d)
        chip_co.append(chip_co_d)
        chip_sig.append(chip_sig_d)

#### Reading in ATAC-seq data

In [ ]:
atac_chr = []
atac_co = []
atac_sig = []
with open(atac_seq_data, "r") as in_file:
    for line in in_file:
        line_array = line.strip().split("\t")
        chr = line_array[0]
        start = int(line_array[1])
        stop = int(line_array[2])
        sig = float(line_array[6])
        
        atac_chr.append(chr)
        atac_co.append([start, stop])
        atac_sig.append(sig)

#### Creating corresponding arrays out of TAD, ChIP-seq and ATAC-seq data

In [ ]:
# Convert to 10 kb bins  
tad_boundary_array = np.zeros(num_bins)
atac_array = np.zeros(num_bins)
atac_array_count = np.zeros(num_bins)
chip_array = np.zeros((gene_count,num_bins))
chip_array_count = np.zeros((gene_count,num_bins))

# Tad boundary data
if 'GM12878' in true_tad_boundaries:
    for tad_i in range(len(tad_chr)):
        chr = tad_chr[tad_i]
        if chr == "chrX":
            continue
        prev_size = chromosome_size[chr][1]
        b1_0 = int((tad_bo_co[tad_i][0] + prev_size) / bin_size)
        b1_1 = int((tad_bo_co[tad_i][1] + prev_size) / bin_size)
        b2_0 = int((tad_bo_co[tad_i][2] + prev_size) / bin_size)
        b2_1 = int((tad_bo_co[tad_i][3] + prev_size) / bin_size)

        if tad_bo_co[tad_i][0] > 0:
            tad_boundary_array[b1_0:b1_1+1] = 1

        if tad_bo_co[tad_i][2] > 0:
            tad_boundary_array[b2_0:b2_1+1] = 1
else:
    for tad_i in range(len(tad_chr)):
        chr = tad_chr[tad_i]
        if chr == "chrX":
            continue
        prev_size = chromosome_size[chr][1]
        b1_0 = int((tad_bo_co[tad_i][0] + prev_size) / bin_size)
        b1_1 = int((tad_bo_co[tad_i][1] + prev_size) / bin_size)

        if tad_bo_co[tad_i][0] > 0:
            tad_boundary_array[b1_0:b1_1+1] = 1
            
# Atac-seq data
for atac_i in range(len(atac_chr)):
    chr = atac_chr[atac_i].strip()
    if chr in chromosome_size:
        prev_size = chromosome_size[chr][1]
    else:
        continue
    c0 = int((atac_co[atac_i][0] + prev_size) / bin_size)
    c1 = int((atac_co[atac_i][1] + prev_size) / bin_size)
    
    sig = atac_sig[atac_i]
    
    atac_array[c0:c1+1] += sig
    atac_array_count[c0:c1+1] += 1

if 'GM12878' in true_tad_boundaries:
    atac_sig_mean = np.divide(atac_array, atac_array_count,where = atac_array_count != 0)
else:
    atac_sig_mean = np.divide(atac_array, atac_array_count, out=np.zeros(len(atac_array)),where = atac_array_count != 0)
    
# Chip-seq data
chip_sig_mean=[]
for count,j in enumerate(chip_chr):
    print(count)
    for chip_i in range(len(j)):
        chr = j[chip_i].strip()
        if chr in chromosome_size:
            prev_size = chromosome_size[chr][1]
        else:
            continue

        c0 = int((chip_co[count][chip_i][0] + prev_size) / bin_size)
        c1 = int((chip_co[count][chip_i][1] + prev_size) / bin_size)

        sig = chip_sig[count][chip_i]
        chip_array[count,c0:c1+1] += sig
        chip_array_count[count,c0:c1+1] += 1
        chip_sig_mean = np.divide(chip_array, chip_array_count, where = chip_array_count != 0)


plt.plot(chip_sig_mean)
plt.show()

In [ ]:
#Writing resulting ChIP-seq 2D array into file

trans=chip_sig_mean.transpose()
inter_matrix=np.hstack((trans,atac_sig_mean.reshape(-1,1)))
final_matrix=np.hstack((inter_matrix,tad_boundary_array.reshape(-1,1)))
np.savetxt(chip_seq_output,final_matrix,delimiter='\t')

In [ ]:
#Loading data matrix for analysis and ML algorithms

data = np.loadtxt(chip_seq_output, delimiter="\t")
num_features=gene_count+2
y = data[:,num_features-1].astype(int)
x = data[:,:num_features-1]

### PCA

In [ ]:
def reduce_dimensionality_pca(X, n_components = 2):
    """Train a PCA model and use it to reduce the training and testing data.
    
    Args:
        filtered_train_gene_expression (n_train x num_top_genes matrix): input filtered training expression data 
        filtered_test_gene_expression (n_test x num_top_genes matrix): input filtered test expression data 
        
    Return:
        (reduced_train_data, reduced_test_data): a tuple of
            1. The filtered training data transformed to the PC space.
            2. The filtered test data transformed to the PC space.
    """
 
    
    pca_train = PCA(n_components=2)
    pca_train.fit(X)
    # pca_train.fit(filtered_test_gene_expression)

    transformed_X = pca_train.transform(X)

    return transformed_X

def plot_transformed_cells(reduced_train_data, train_labels):
    """Plot the PCA-reduced training data using just the first 2 principal components.
    
    Args:
        reduced_train_data (n_train x num_components matrix): reduced training expression data
        train_labels (array of length n_train): array of cell type labels for training data
        
    Return:
        None

    """
    
    pca_df = pd.DataFrame()
    pca_df['PC1'] = reduced_train_data[:, 0]
    pca_df['PC2'] = reduced_train_data[:, 1]
   
    new_labels=[]
    for i in train_labels:
        if i==0:
            new_labels.append('No TAD Boundary')
        else:
            new_labels.append('TAD Boundary')
    pca_df['Labels'] = new_labels
    #sns.set()
    sns.set_style("white")
    sns.lmplot(
    x='PC1', 
    y='PC2', 
    data=pca_df, 
    hue='Labels', 
    fit_reg=False, 
    legend=True
    )
    
    ax = plt.gca()
    ax.set_title("Data visualisation with PCA",fontsize=16)
    #plt.savefig('/Users/sanatmishra27/Downloads/CompGenomics/02710_Project/Results/PCA.png',dpi=300,bbox_inches='tight')
    plt.show()
    
top_components=reduce_dimensionality_pca(x)
plot_transformed_cells(top_components, y)

### GBM (Gradient Boosting Machine)

In [ ]:
#GBM
# Runs ML model on data

num_tad_pts = np.sum(y)
num_pts = x.shape[0]

tb_data = np.zeros((num_tad_pts,num_features))
non_tb_data = np.zeros((num_pts - num_tad_pts,num_features))

tb_i = 0
non_i = 0
for i in range(num_pts):
    if data[i,num_features-1] == 1:
        tb_data[tb_i,:] = data[i,:]
        tb_i += 1
    else:
        non_tb_data[non_i,:] = data[i,:]
        non_i += 1

gen = np.random.default_rng(1)
gen.shuffle(tb_data, 0)
gen.shuffle(non_tb_data,0)

num_tad_pts = tb_data.shape[0]
data_stop = int(.7*num_tad_pts)
train_tb_data = tb_data[:data_stop,:]
test_tb_data = tb_data[data_stop:,:]
train_non_tb_data = non_tb_data[:data_stop,:]
test_non_tb_data = non_tb_data[data_stop:2*data_stop,:]

train_data = np.vstack((train_tb_data,train_non_tb_data))
test_data = np.vstack((test_tb_data, test_non_tb_data))

#Predict using GBM
fit = GradientBoostingClassifier().fit(train_data[:,:num_features-1], train_data[:,num_features-1])
pred = fit.predict(test_data[:,:num_features-1])

#Evaluate model

aucs=[]
precisions=[]
recalls=[]
f1s=[]
accuracy=[]

accuracy.append(accuracy_score(test_data[:,num_features-1],pred))
precisions.append(precision_score(test_data[:,num_features-1],pred))
recalls.append(recall_score(test_data[:,num_features-1],pred))
aucs.append(roc_auc_score(test_data[:,num_features-1],pred))
f1s.append(f1_score(test_data[:,num_features-1],pred))

### Feature Importances

In [ ]:
import matplotlib.pyplot as plt

plt.bar(range(len(fit.feature_importances_)), sorted(fit.feature_importances_,reverse=True))
plt.ylabel('Scaled importance')
if 'GM12878' in true_tad_boundaries:
    label=np.array(['RAD21', 'ZNF143', 'SMC3', 'YY1', 'JUND', 'EP300', 'CTCF', 'ATAC-Seq'])
    label=label[np.argsort(fit.feature_importances_)[::-1]]
    plt.xticks(range(len(fit.feature_importances_)),label,rotation='vertical')
    plt.title('Feature Importance summary for GM12878')
    plt.grid(False)
    
else:

    label=np.array(['CTCF', 'EP300', 'JUND','ATAC-Seq','RAD21'])
    label=label[np.argsort(fit.feature_importances_)[::-1]]
    plt.xticks(range(len(fit.feature_importances_)),label,rotation='vertical')
    plt.title('Feature Importance summary for MCF7')
    
#plt.savefig('/Users/sanatmishra27/Downloads/CompGenomics/02710_Project/Results/FeatureImportance.pdf',transparent=True,bbox_inches='tight')
plt.show()

### Decision Tree

In [ ]:
#Decision Tree

rf = RandomForestClassifier()
rf.fit(train_data[:,:num_features-1], train_data[:,num_features-1])
y_pred = rf.predict(test_data[:,:num_features-1])

accuracy.append(accuracy_score(test_data[:,num_features-1],y_pred))
precisions.append(precision_score(test_data[:,num_features-1],y_pred))
recalls.append(recall_score(test_data[:,num_features-1],y_pred))
aucs.append(roc_auc_score(test_data[:,num_features-1],y_pred))
f1s.append(f1_score(test_data[:,num_features-1],y_pred))

### Plotting model performances

In [ ]:
labels = ['Gradient Boost',"Random Forest"]#, "SVM"]
x = np.arange(len(labels))  # the label locations
width = 0.2  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - 1.5*width, precisions, width, label='Precision')
rects2 = ax.bar(x - .5*width, recalls, width, label='Recall')
rects3 = ax.bar(x + .5*width, f1s, width, label='F1')
rects4 = ax.bar(x + 1.5*width, accuracy, width, label='Accuracy')

plt.ylabel('Performance Value')
plt.title('Model Comparison')
plt.xticks(x, labels)
plt.legend(bbox_to_anchor=(1, 1.05))


fig.tight_layout()
plt.grid(False)
#plt.savefig('/Users/sanatmishra27/Downloads/CompGenomics/02710_Project/Results/ModelPerformance.png',transparent=True,dpi=300,bbox_inches='tight')
plt.show()